## モジュール読み込み・データセットアップ

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# transformersをインストール
!pip install transformers
!pip install mecab-python3==0.996.2 fugashi ipadic
from transformers import BertJapaneseTokenizer, BertModel

## データの確認

In [ ]:
review_df = pd.read_csv("review_df_yahoo_tokyo.csv")
review_df

,Unnamed: 0,Uid,Name,Id,Body,Subject,Guid,Author,Rating,VisitDate,Scene,UsefulCount,LinkUrl,CreateDate,UpdateDate
0,0,03b5ffb0e26db0f405da03edde8b47542ede27d6,浅草寺,f30c52e8131654d568ecc8be77e9b84d41e32b57,東京にきたら絶対によります♪<br />日中の賑わっている時もいいですが、早朝や雨が降ってい...,東京きたら絶対よります♪,AI3755HMEQFF3EASJ3LGV7E57A,ゆう,3.0,NaN,NaN,NaN,NaN,2008-07-16T15:22:41+09:00,2015-02-20T05:08:04+09:00
1,1,d8160ecdbf9163a8100c643838d956e9780e64b9,レゴランド・ディスカバリー・センター東京,dd479a6937ecf4da36183e00b30beacd3443850d,平日子供の学校が創立記念日だったり、振り替え休日だったので、行ってきました。\n事前予約をす...,子供の頃の夢があります,3NYGP5OMBJJAMQM22JFKNSWHDE,えもえもん,4.0,2013-02-01T00:00:00+09:00,3.0,1.0,NaN,2013-02-17T21:35:28+09:00,2015-02-20T00:29:32+09:00
2,2,2e995a38cf30d8b72839edb8d44e47b9b710d091,株式会社東京タワー,f2aad12f72fad0dcbeebb7e361ed042b882a0ec5,夜景とかやばいですからね?<br /><br />東京中がギラギラギラギラに光ってて～～～～...,夜景が☆,TXIATK2EHPLZIZHDAA4YFNDYJU,ありおかゆうり,3.0,NaN,NaN,NaN,NaN,2010-03-08T19:44:13+09:00,2015-02-19T14:17:05+09:00
3,3,620c1e3a98595ca9436e192203d12d7460ae95d3,味中味,84e11997db796b6c3d51c077d7dcca9c4dc44ea8,メニューにはずれがありませんでした。\r\n餃子も最高です。\r\n最近、出前が多いようで ...,中華街に行く必要がなくなりました。,NaN,NaN,5.0,NaN,3.0,NaN,NaN,2011-02-15T00:36:08+09:00,2011-02-15T00:36:08+09:00
4,4,f336e3170f0a3b88ce04002f7d4c842a6d6fabbb,日本空港ビルデング株式会社羽田空港国内線総合案内,5710cd6155e3a3ab176238ea61990e3f2ede3322,夜景という観点で口コミさせて頂きます。\n\n５年ほど前から、夏場の夜に何回か展望デッキから...,展望デッキからの夜景がキレイ,F7W27HVEY6TW5Z2TIDS3RGTWJU,謎のα,4.0,2010-10-01T00:00:00+09:00,NaN,NaN,NaN,2011-06-29T14:32:11+09:00,2015-02-20T08:24:54+09:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5552,78bea50af99a9e6b17e065509475f1c9687f13f6,ギンレイホール,7abd9cba4ea05a73395d90c2f6dc65d96e7a1bc2,客のマナーが非常に悪い。<br />上映中に飲食をする客がコンビニの袋を開いたり閉じたりして...,客のマナーが気になる,3WGZEAWLNQZ5H2TDLTF7VMQN7A,ttm*zu,3.0,NaN,NaN,NaN,NaN,2009-11-12T14:08:55+09:00,2015-02-20T10:10:47+09:00
5566,5553,0c3767083350dc7631e9f774f54c975faf1f7344,かごの屋 三鷹野崎店,5e4f45985cdb39a5b1d05af2c4196231666a5a1c,食べ放題を頼んだけど 頼んでないものが運ばれてきたり 頼んだものが来なかったり 忙しかったん...,NaN,BJLZUGLMO72KXVMBXLZB3A5VFQ,lal*****,3.0,NaN,NaN,NaN,NaN,2020-11-07T09:12:13+09:00,2020-11-15T00:44:54+09:00
5567,5554,52279dcf5bef6f304c3a52a0c9e0ec28b6ada6f0,焼肉やまと船橋本店,75a26bbec2be49368d06747966f7a3dd53c70908,辛口になりますが値段が高いですね。（不相応だと）\r\n\r\n金額と品質がともなっていない...,うーん、もういかない。,NaN,NaN,2.0,NaN,2.0,3.0,NaN,2009-10-06T19:36:26+09:00,2012-09-12T09:13:01+09:00
5568,5555,5f7c1babe29f0737cece48845e5487a84cccb21b,El Castellano,d2092624f82148e30735c61f6af1b99f8d0be95e,遅れて来た友人を待つのにオーダーすると\r\n前菜のみとの答え、揃ってからオーダーを\r\n...,だけど美味しい,NaN,NaN,5.0,NaN,2.0,NaN,NaN,2008-05-22T13:41:36+09:00,2010-08-23T13:57:16+09:00


In [ ]:
# Rating>3ならpositive、Rating<3ならnegativeとする。Rating==3は使用しない。
def Ratingmap(x):
    if x > 3:
        return 1
    elif x < 3:
        return 0
    else:
        return -1
train_df = pd.DataFrame({"id":review_df["Id"],
                         "text":review_df["Body"],
                         "label":review_df["Rating"].map(Ratingmap)})
train_df = train_df[train_df["label"]!= -1].copy()

In [ ]:
train_df.head(5)

,id,text,label
1,dd479a6937ecf4da36183e00b30beacd3443850d,平日子供の学校が創立記念日だったり、振り替え休日だったので、行ってきました。\n事前予約をす...,1
3,84e11997db796b6c3d51c077d7dcca9c4dc44ea8,メニューにはずれがありませんでした。\r\n餃子も最高です。\r\n最近、出前が多いようで ...,1
4,5710cd6155e3a3ab176238ea61990e3f2ede3322,夜景という観点で口コミさせて頂きます。\n\n５年ほど前から、夏場の夜に何回か展望デッキから...,1
6,1193920f64b4f6fcea9d5d6dfde81418acec21b0,ここのメインダイニング、グランカフェは朝食、昼食、夜食を何回か利用しましたが\r\nあまり美...,0
11,a21f2e4eabb2c9b188800a42e34b3198319a997f,こう総会の海側の部屋でしたがとにかく眺めが良い。食事も満足しました。夕食時の席は横浜スタジア...,1


In [ ]:
train_df["label"].value_counts()

1    2971
0    1263
Name: label, dtype: int64

In [ ]:
# tokenizerの定義とvocabの辞書
model_name = 'cl-tohoku/bert-base-japanese'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
for i,item in enumerate(tokenizer.vocab.items()):
  print(item)
  if i > 100:
    break

('[PAD]', 0)
('[UNK]', 1)
('[CLS]', 2)
('[SEP]', 3)
('[MASK]', 4)
('の', 5)
('、', 6)
('に', 7)
('。', 8)
('は', 9)
('た', 10)
('を', 11)
('で', 12)
('と', 13)
('が', 14)
('し', 15)
('て', 16)
('1', 17)
('な', 18)
('年', 19)
('れ', 20)
('い', 21)
('あ', 22)
('(', 23)
(')', 24)
('2', 25)
('さ', 26)
('こ', 27)
('も', 28)
('か', 29)
('##する', 30)
('ある', 31)
('日', 32)
('いる', 33)
('する', 34)
('・', 35)
('「', 36)
('月', 37)
('」', 38)
('19', 39)
('から', 40)
('20', 41)
('大', 42)
('ア', 43)
('そ', 44)
('こと', 45)
('##して', 46)
('ま', 47)
('3', 48)
('や', 49)
('として', 50)
('中', 51)
('一', 52)
('人', 53)
('よ', 54)
('ス', 55)
('によ', 56)
('4', 57)
('なっ', 58)
('その', 59)
('ら', 60)
('-', 61)
('れる', 62)
('『', 63)
('など', 64)
('』', 65)
('フ', 66)
('シ', 67)
('##リー', 68)
('同', 69)
('この', 70)
('出', 71)
('時', 72)
('お', 73)
('地', 74)
('だ', 75)
('5', 76)
('行', 77)
('201', 78)
('国', 79)
('ない', 80)
('的', 81)
('ため', 82)
('後', 83)
('られ', 84)
('発', 85)
('200', 86)
('##ール', 87)
('イ', 88)
('##ラン', 89)
('作', 90)
('日本', 91)
('##ター', 92)
('##ック', 93)
('市',

In [ ]:
pad = tokenizer.vocab["[PAD]"]
max_lengths=512

In [ ]:
# tokenizerの分割例
text = "カーネーションが綺麗だった"
tokenizer.tokenize(text)

['カーネ', '##ーション', 'が', '綺麗', 'だっ', 'た']

## input_dataの作成

In [ ]:
# tokenをid化
token = ["[CLS]"]+tokenizer.tokenize(text)[:max_lengths-2]+["[SEP]"]
input_id = tokenizer.convert_tokens_to_ids(token)
# 1文のみなのでsegment_idは全て0
segment_id = [0]*max_lengths
# input_idが存在する箇所に1を立てる
attention_mask = [1]*len(input_id)+[0]*(max_lengths - len(input_id))
input_id = input_id+[pad]*(max_lengths-len(input_id))

In [ ]:
print("text:",text)
print("token:",token)
print("input_id:",input_id[:15],"len:",len(input_id))
print("segment_id:",segment_id[:15],"len:",len(segment_id))
print("attention_mask:",attention_mask[:15],"len:",len(attention_mask))

text: カーネーションが綺麗だった
token: ['[CLS]', 'カーネ', '##ーション', 'が', '綺麗', 'だっ', 'た', '[SEP]']
input_id: [2, 22377, 950, 14, 27344, 308, 10, 3, 0, 0, 0, 0, 0, 0, 0] len: 512
segment_id: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] len: 512
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] len: 512


In [ ]:
# 上記のinput_dataの作成方法をもとにdataset化
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels=[]):
        self.input_ids, self.segment_ids, self.attention_masks = [],[],[]
        for text in tqdm(texts):
          token = ["[CLS]"]+tokenizer.tokenize(text)[:max_lengths-2]+["[SEP]"]
          input_id = tokenizer.convert_tokens_to_ids(token)
          segment_id = [0]*max_lengths
          attention_mask = [1]*len(input_id)+[0]*(max_lengths - len(input_id))
          input_id = input_id+[pad]*(max_lengths-len(input_id))
          self.input_ids.append(input_id)
          self.segment_ids.append(segment_id)
          self.attention_masks.append(attention_mask)
        self.input_ids = np.array(self.input_ids)
        self.segment_ids = np.array(self.segment_ids)
        self.attention_masks = np.array(self.attention_masks)
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
      if len(self.labels):
        return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx], self.labels[idx]
      else:
        return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx]


In [ ]:
# datasetとdataloaderの作成
from sklearn.model_selection import train_test_split
batch_size=8
X,y = train_df["text"].values, train_df["label"].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=100,stratify=y)
train_ds = ReviewDataset(texts=X_train, labels=y_train)
train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=batch_size, shuffle=True)  
val_ds = ReviewDataset(texts=X_val, labels=y_val)
val_dl = torch.utils.data.DataLoader(
    val_ds, batch_size=batch_size, shuffle=False)

In [ ]:
# Bertの事前学習済みエンコーダーを使用して2値クラス分類器を作成

class BertClassification(nn.Module):
  def __init__(self, bert):
      super(BertClassification, self).__init__()

      # BERTモジュール
      self.bert = bert  # BERTモデル

      self.cls = nn.Linear(in_features=768, out_features=2)

      # 重み初期化処理
      nn.init.normal_(self.cls.weight, std=0.02)
      nn.init.normal_(self.cls.bias, 0)

  def forward(self, input_ids, token_type_ids=None, attention_mask=None):

      # BERTの基本モデル部分の順伝搬
      # 順伝搬させる
    output = self.bert(
              input_ids, token_type_ids, attention_mask)
    # [CLS]に対応する特徴量を取得します。
    pooled_output = output[1]

    # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
    pooled_output = pooled_output.view(-1, 768)  # sizeを[batch_size, hidden_sizeに変換
    out = self.cls(pooled_output)

    return out

## 学習

In [ ]:
cuda = torch.cuda.is_available()
# 学習済みモデルをダウンロード
bert = BertModel.from_pretrained(model_name)
model =  BertClassification(bert)
# 高速化
torch.backends.cudnn.benchmark = True
if cuda:
  model.cuda()
# optimizerの設定
optimizer = optim.Adam(model.parameters(),lr = 4e-4 ,betas=(0.9, 0.999))

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

In [ ]:
cuda

True

In [ ]:
# 最後以外のBertLayerモジュールのパラメータを固定(実際のタスクでは固定させないことも多い)
for param in model.bert.encoder.layer[:-1].parameters():
    param.requires_grad = False

In [ ]:
## 学習します。1epochあたり約8分です。
epochs = 2
for epoch in range(epochs):
  total_loss = 0
  accuracy = 0
  model.train()
  
  print("epoch {} start!".format(epoch+1))
  # train
  for iter_num, (input_ids, segment_ids, attention_masks, labels) in tqdm(enumerate(train_dl),total = len(train_dl)):
    optimizer.zero_grad()
    if cuda:
      input_ids, segment_ids, attention_masks, labels =\
      input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
    # forward(順伝搬)
    outputs = model(input_ids = input_ids,
                    token_type_ids = segment_ids,
                    attention_mask = attention_masks)
    pred_proba = outputs.softmax(dim=1)[:,1]
    pred = (pred_proba>=0.5).type(torch.int)
    loss = criterion(outputs,labels)

    # backward(逆伝搬)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    accuracy += (pred==labels).sum().item()
    #50 iterごとにlossとaccuracyを表示
    if (iter_num+1) % 50 == 0:
      size = batch_size*(iter_num+1)
      print("{} iter loss:{:.4f} accuracy:{:.4f}".format(
          iter_num+1,total_loss/(iter_num+1),accuracy/size))


  total_loss /= len(train_dl)
  accuracy /= len(train_ds)

  # validation
  val_total_loss = 0
  val_accuracy = 0
  model.eval()
  for input_ids, segment_ids, attention_masks, labels in tqdm(val_dl):
    if cuda:
      input_ids, segment_ids, attention_masks, labels =\
      input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda(), labels.cuda()
    with torch.no_grad():
      outputs = model(input_ids = input_ids,
                      token_type_ids = segment_ids,
                      attention_mask = attention_masks)
      pred_proba = outputs.softmax(dim=1)[:,1]
      pred = (pred_proba>=0.5).type(torch.int)
      loss = criterion(outputs,labels)
      val_total_loss += loss.item()
      val_accuracy += (pred==labels).sum().item()

  val_total_loss /= len(val_dl)
  val_accuracy /= len(val_ds)
  print("epoch{} total loss:{:.4f}, accuracy:{:.4f}, val_total loss:{:.4f}, val_accuracy:{:.4f}"\
        .format(epoch+1,total_loss,accuracy,val_total_loss,val_accuracy))
torch.save(model.state_dict(), './model.hdf5')

epoch 1 start!


50 iter loss:0.6449 accuracy:0.6725
100 iter loss:0.5442 accuracy:0.7375
150 iter loss:0.5069 accuracy:0.7700
200 iter loss:0.4714 accuracy:0.7925
250 iter loss:0.4588 accuracy:0.8050
300 iter loss:0.4354 accuracy:0.8154
350 iter loss:0.4166 accuracy:0.8239




epoch1 total loss:0.4073, accuracy:0.8292, val_total loss:0.2389, val_accuracy:0.9056
epoch 2 start!


50 iter loss:0.1652 accuracy:0.9550
100 iter loss:0.1975 accuracy:0.9437
150 iter loss:0.2088 accuracy:0.9392
200 iter loss:0.1998 accuracy:0.9381
250 iter loss:0.1991 accuracy:0.9380
300 iter loss:0.1959 accuracy:0.9367
350 iter loss:0.1894 accuracy:0.9396




epoch2 total loss:0.1871, accuracy:0.9396, val_total loss:0.1697, val_accuracy:0.9315


## 予測

モデルのロード

In [ ]:
model.load_state_dict(torch.load("./model.hdf5"))

<All keys matched successfully>

テスト

In [ ]:
X_test = ["今日のランチは最高だった！"]
test_ds = ReviewDataset(texts=X_test)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=1, shuffle=False)

In [ ]:
for input_ids, segment_ids, attention_masks in test_dl:
  if cuda:
    input_ids, segment_ids, attention_masks =\
      input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda()
  outputs = model(input_ids = input_ids,
                  token_type_ids = segment_ids,
                  attention_mask = attention_masks)
  pred_proba = outputs.softmax(dim=1)[:,1]
  print("入力テキストのポジティブ度: {:.4f}".format(pred_proba.item()))

入力テキストのポジティブ度: 0.9835


テスト2

In [ ]:
X_test = ["今日のランチは微妙だった。"]
test_ds = ReviewDataset(texts=X_test)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=1, shuffle=False)

In [ ]:
for input_ids, segment_ids, attention_masks in test_dl:
  if cuda:
    input_ids, segment_ids, attention_masks =\
      input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda()
  outputs = model(input_ids = input_ids,
                  token_type_ids = segment_ids,
                  attention_mask = attention_masks)
  pred_proba = outputs.softmax(dim=1)[:,1]
  print("入力テキストのポジティブ度: {:.4f}".format(pred_proba.item()))

入力テキストのポジティブ度: 0.0237
